In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

In [2]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        '''                                                                      
        Args:                                                                    
          input_dim, hidden_dim, output_dim (int): number of perceptrons in 
            each layer (only one hidden layer).                                       
        '''
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        '''                                                                      
        Forward Pass                                                             
        Args:                                                                    
          x_in: (torch.Tensor): input tensor, .shape should be                   
            (batch, input_dim)                                                   
          apply_softmax (bool): apply softmax? F if used with x-entropy          
        Returns:                                                                 
          tensor with shape (batch, output_dim)                                  
        '''
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

In [3]:
BATCH = 2
INPUT = 3
HIDDEN = 100
OUTPUT = 4

mlp = MultilayerPerceptron(INPUT, HIDDEN, OUTPUT)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


In [6]:
def describe(x):
    print(f'Type: {x.type()}')
    print(f'Shape: {x.shape}')
    print(f'Values:\n{x}')

In [9]:
x_input = torch.rand(BATCH, INPUT)
describe(x_input)

Type: torch.FloatTensor
Shape: torch.Size([2, 3])
Values:
tensor([[0.6213, 0.5737, 0.4899],
        [0.2528, 0.3296, 0.7179]])


In [10]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

Type: torch.FloatTensor
Shape: torch.Size([2, 4])
Values:
tensor([[ 0.0587, -0.1002, -0.0899,  0.1464],
        [ 0.0337, -0.1321,  0.0457,  0.0584]], grad_fn=<ThAddmmBackward>)


In [11]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

Type: torch.FloatTensor
Shape: torch.Size([2, 4])
Values:
tensor([[0.2627, 0.2241, 0.2264, 0.2868],
        [0.2575, 0.2181, 0.2606, 0.2639]], grad_fn=<SoftmaxBackward>)


In [12]:
y_output.sum()

tensor(2., grad_fn=<SumBackward0>)

In [15]:
class SurnameDataset(Dataset):
    def __getitem__(self, idx):
        row = self._target_df.iloc[idx]
        surname_vector = self._vectorizer.vectorize(row.surname)
        nationality_idx = self._vectorizer.nationality_vocab.lookup_token(
            row.nationality)
        return {'x_surname': surname_vector, 
                'y_nationality': nationality_idx}